# 学習済みのT5を使って日本語要約生成モデルを構築

## 準備

### Googleドライブのマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Googleドライブ直下に「text_generation」フォルダーを作り、そのフォルダーで作業する

In [2]:
import os
path = '/content/drive/MyDrive/google_colaboratory/document_summarization/'
if not os.path.exists(path):
    os.makedirs(path)

In [3]:
%cd /content/drive/MyDrive/google_colaboratory/document_summarization/

/content/drive/MyDrive/google_colaboratory/document_summarization


下記のコードで、割り当てられたGPUの使用可能容量を確認している。

In [4]:
!pip install gputil
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=44b2b807603cc69d0531bdfa5ad9375f6fc8c01c9feb93f1eae13fe5a40474ae
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Gen RAM Free: 26.3 GB  | Proc size: 117.8 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


### モジュールのインポート

In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.6 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
     |████████████████████████████████| 3.3 MB 54.7 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172979 sha256=b75d90040a8275f68ccf1817af5c492dff27194ba072367922e7cd2d6c7dc01d
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn


In [6]:
from pathlib import Path
import re
import math
import time
import copy
from tqdm import tqdm
import pandas as pd
import tarfile
import neologdn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
import settings

In [7]:
%load_ext autoreload
%autoreload 2
pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)
pd.set_option('max_colwidth', 300)

## パラメータの設定

In [8]:
data_dir_path = Path('data')

## データの取得

In [9]:
body_data = pd.read_csv(data_dir_path.joinpath('body_data.csv'))
summary_data = pd.read_csv(data_dir_path.joinpath('summary_data.csv'))

## 学習データの作成

### データの前処理

In [10]:
def join_text(x, add_char='。'):
    return add_char.join(x)

In [11]:
def preprocess_text(text):
    text = re.sub(r'[\r\t\n\u3000]', '', text)
    text = neologdn.normalize(text)
    text = text.lower()
    text = text.strip()
    return text

In [12]:
summary_data = summary_data.query('text.notnull()', engine='python').groupby(
    'article_id'
).agg({'text': join_text})

In [13]:
body_data = body_data.query('text.notnull()', engine='python')

In [14]:
data = pd.merge(
    body_data.rename(columns={'text': 'body_text'}),
    summary_data.rename(columns={'text': 'summary_text'}),
    on='article_id', how='inner'
).assign(
    body_text=lambda x: x.body_text.map(lambda y: preprocess_text(y)),
    summary_text=lambda x: x.summary_text.map(lambda y: preprocess_text(y))
)

### データを学習用データと確認用データに分離し、ベクトル化

In [15]:
def convert_batch_data(train_data, valid_data, tokenizer):

    def generate_batch(data):

        batch_src, batch_tgt = [], []
        for src, tgt in data:
            batch_src.append(src)
            batch_tgt.append(tgt)

        batch_src = tokenizer(
            batch_src, max_length=settings.max_length_src, truncation=True, padding="max_length", return_tensors="pt"
        )
        batch_tgt = tokenizer(
            batch_tgt, max_length=settings.max_length_target, truncation=True, padding="max_length", return_tensors="pt"
        )

        return batch_src, batch_tgt

    train_iter = DataLoader(train_data, batch_size=settings.batch_size_train, shuffle=True, collate_fn=generate_batch)
    valid_iter = DataLoader(valid_data, batch_size=settings.batch_size_valid, shuffle=True, collate_fn=generate_batch)

    return train_iter, valid_iter

In [16]:
tokenizer = T5Tokenizer.from_pretrained(settings.MODEL_NAME, is_fast=True)

Downloading:   0%|          | 0.00/804k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    data['body_text'], data['summary_text'], test_size=0.2, random_state=42, shuffle=True
)

train_data = [(src, tgt) for src, tgt in zip(X_train, y_train)]
valid_data = [(src, tgt) for src, tgt in zip(X_test, y_test)]

train_iter, valid_iter = convert_batch_data(train_data, valid_data, tokenizer)

## ニュースタイトルから本文を生成するテキスト生成モデルの構築

### モデル定義クラス

In [18]:
class T5FineTuner(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.model = T5ForConditionalGeneration.from_pretrained(settings.MODEL_NAME)

    def forward(
        self, input_ids, attention_mask=None, decoder_input_ids=None,
        decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

### モデル学習処理関数

In [19]:
def train(model, data, optimizer, PAD_IDX):
    
    model.train()
    
    loop = 1
    losses = 0
    pbar = tqdm(data)
    for src, tgt in pbar:
                
        optimizer.zero_grad()
        
        labels = tgt['input_ids'].to(settings.device)
        labels[labels[:, :] == PAD_IDX] = -100

        outputs = model(
            input_ids=src['input_ids'].to(settings.device),
            attention_mask=src['attention_mask'].to(settings.device),
            decoder_attention_mask=tgt['attention_mask'].to(settings.device),
            labels=labels
        )
        loss = outputs['loss']

        loss.backward()
        optimizer.step()
        losses += loss.item()
        
        pbar.set_postfix(loss=losses / loop)
        loop += 1
        
    return losses / len(data)

In [20]:
def evaluate(model, data, PAD_IDX):
    
    model.eval()
    losses = 0
    with torch.no_grad():
        for src, tgt in data:

            labels = tgt['input_ids'].to(settings.device)
            labels[labels[:, :] == PAD_IDX] = -100

            outputs = model(
                input_ids=src['input_ids'].to(settings.device),
                attention_mask=src['attention_mask'].to(settings.device),
                decoder_attention_mask=tgt['attention_mask'].to(settings.device),
                labels=labels
            )
            loss = outputs['loss']
            losses += loss.item()
        
    return losses / len(data)

### モデルの学習

In [21]:
model = T5FineTuner()
model = model.to(settings.device)

optimizer = optim.Adam(model.parameters())

PAD_IDX = tokenizer.pad_token_id
best_loss = float('Inf')
best_model = None
counter = 1

for loop in range(1, settings.epochs + 1):

    start_time = time.time()

    loss_train = train(model=model, data=train_iter, optimizer=optimizer, PAD_IDX=PAD_IDX)

    elapsed_time = time.time() - start_time

    loss_valid = evaluate(model=model, data=valid_iter, PAD_IDX=PAD_IDX)

    print('[{}/{}] train loss: {:.4f}, valid loss: {:.4f} [{}{:.0f}s] counter: {} {}'.format(
        loop, settings.epochs, loss_train, loss_valid,
        str(int(math.floor(elapsed_time / 60))) + 'm' if math.floor(elapsed_time / 60) > 0 else '',
        elapsed_time % 60,
        counter,
        '**' if best_loss > loss_valid else ''
    ))

    if best_loss > loss_valid:
        best_loss = loss_valid
        best_model = copy.deepcopy(model)
        counter = 1
    else:
        if counter > settings.patience:
            break

        counter += 1

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

100%|██████████| 1346/1346 [15:39<00:00,  1.43it/s, loss=2.07]


[1/1000] train loss: 2.0740, valid loss: 1.8215 39s] counter: 1 **


100%|██████████| 1346/1346 [16:07<00:00,  1.39it/s, loss=1.68]


...

[20/1000] train loss: 0.1993, valid loss: 3.2527 10s] counter: 19 


100%|██████████| 1346/1346 [16:10<00:00,  1.39it/s, loss=0.192]


[21/1000] train loss: 0.1923, valid loss: 3.3627 11s] counter: 20 


100%|██████████| 1346/1346 [16:10<00:00,  1.39it/s, loss=0.18]


[22/1000] train loss: 0.1800, valid loss: 3.3809 11s] counter: 21 


## モデルの保存

In [22]:
model_dir_path = Path('model')
if not model_dir_path.exists():
    model_dir_path.mkdir(parents=True)

In [23]:
tokenizer.save_pretrained(model_dir_path)
best_model.model.save_pretrained(model_dir_path)

In [24]:
tokenizer.save_pretrained(model_dir_path)
best_model.model.save_pretrained(model_dir_path)

## 学習したモデルを使って文章から要約を生成

In [ ]:
def generate_text_from_model(title, trained_model, tokenizer, num_return_sequences=1):

    trained_model.eval()
    
    title = preprocess_text(title)
    batch = tokenizer(
        [title], max_length=settings.max_length_src, truncation=True, padding="longest", return_tensors="pt"
    )

    # 生成処理を行う
    outputs = trained_model.generate(
        input_ids=batch['input_ids'].to(settings.device),
        attention_mask=batch['attention_mask'].to(settings.device),
        max_length=settings.max_length_target,
        repetition_penalty=8.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        # temperature=1.0,  # 生成にランダム性を入れる温度パラメータ
        # num_beams=10,  # ビームサーチの探索幅
        # diversity_penalty=1.0,  # 生成結果の多様性を生み出すためのペナルティパラメータ
        # num_beam_groups=10,  # ビームサーチのグループ
        num_return_sequences=num_return_sequences,  # 生成する文の数
    )

    generated_texts = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs
    ]

    return generated_texts

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_dir_path)
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_path)

In [ ]:
index = 0
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
あなたの厄年どでした?

□ 生成本文
みなさんは、年末の大掃除やお正月の準備はしていますか?「大掃除」と聞いて、「今年の厄年は何回したっけ?」という疑問が脳裏をよぎることはありませんか?会社員の美枝子さん(仮名・36歳)も、その時期についてこんな質問をしたんだろうなぁと思ったそうです。「そろそろ梅雨明けまで自分の体調管理に気を配らなくちゃいけない時期に差し掛かりたい!」と思いつつも、忙しくて家に帰れない日々にイライラしたり、仕事で疲れてしまってなかなか乗り切れないのが悩みの種ですね。一年のうち、2月15日くらいは肌寒い日が続きましたね。私は1ヶ月に1回は会社帰りに同僚と3人で手を繋いで出かけたり、色々と手を繋いだりして過ごす人が多かったので、職場の女子たちはそんな厄年を気にしなくていいように、普段から念入りにチェックするように心がけながら過ごしているようです。ちなみに昨年の厄年は『二宮御成婚』という超富裕層向けのサービス業を営む芳子さん。さっそく結婚までの紆余曲折を経て結婚した美枝子さんの家族も、それぞれパターンにあった厄払い法を教えてもらいました。最初のうちは、嫁さんが亡くなった後のことはあまり覚えていなかったので、私も母も慌てて様子見だったのですが...。その後、親戚の家に集まり、厄払いを始めた

□ 教師データ本文
“女30代"=人生の節目、転機、結婚適齢期、そして「厄」に振り回されるお年頃といっても過言ではない。というのも、女の厄年は数え年(※1)で、19歳、33歳(大厄)、37歳(小厄)と30代で大厄・小厄と言われている厄が2回もあり、その厄には前厄と後厄がついてまわる。つまり、30代のうちの6年間は「厄」が事あるごとに脳内を駆け抜けてゆく。厄年の起源は諸説あり、すでに平安時代では、貴族たちの間で厄払いが行われていたらしい。現在の厄年が定着したのは江戸時代と言われていて、特に33歳、37歳というのは女性にとって昔も今も精神面や健康面に変化が生じやすい時期だとされているのだろう。厄年を気にするか気にしないかは人それぞれだが、今回は30代の厄年を終えた女性、只今厄年真っ最中の女性たちに厄年体験談をうかがった。“厄年気にする派"のナオミさん(39歳)は「大厄(32)の時は彼氏の浮気と借金が発覚して、ストレスで血尿と激しい胃痛の末、人生初の胃カメラ

In [ ]:
index = 12
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
あなたが同性から嫌われる理由

□ 生成本文
女の敵は女。職場では上司や先輩との付き合い、結婚をすれば姑・小姑との付き合い、子供ができればママ友との付き合いに悩む女性たちは多い。同性に嫌われては平和な日常生活を過ごすことができないといっても過言ではない。嫌われないようにするにはどうすればいいのか。まずは周りの独女たちにどんな女性が嫌いかを聞いてみた。○性格がちょっとイチャメチャで、男性に対しては警戒心が強い。○普段は自分より何年も若く見られている気がするのだが、なぜかいつも自分が本気で自分を省みようとするところがある。○普段から目につく異性に対して態度ががらりと変わってぶりっこになる。○外見とは裏腹に他人を見下す天然な女性のことをいう。○容姿に裏表がある。○身長が低いと相手に好意を見せるときに「好き」と思われがちだが、意外とその人の素顔を知ることができるのだ。○美人で色っぽい女性って、自分に自信を持つことだってできるんだよね。○言動が子供っぽさの中にありながら、自分の話をきちっと話せることが大切。○人の性格と同性から見ても好感度はグンと上がる。同性からの嫌われ女の特徴だが、男女間のギャップが最も大きいだろう。同性の女性は同性がモテる理由のひとつとして「草食系」が多いようだ。同性と接する機会は少なくないが、同性から嫌われることの特徴としては、「社交性が高い女性」である。同性

□ 教師データ本文
女の敵は女。職場ではお局様をはじめ女性社員との付き合い、結婚をすれば姑・小姑との付き合い、子供ができればママ友との付き合いに悩む女性たちは多い。同性に嫌われては平和な日常生活を過ごすことができないといっても過言ではない。嫌われないようにするにはどうすればいいのか。まずは周りの独女たちにどんな女性が嫌いかを聞いてみた。○有名人の友達がいるとか、友達の別荘に招待されたとか、自慢ではないことを自慢する。○男性がいると声のトーンが一オクターブ上がる。○若く見えないのに、自分が若く見られると勘違いしている。○何に自信を持っているか分からないけど絶えず上から目線。○人の批判が多い。○男性の前で態度ががらりと変わってぶりっこになる。○いつも自分が一番でなければ気が済まない、女王様気質。○自分勝手でまわりを振り回す。○性格に裏表がある。○普段はぼっとしているのに、男性の前になる

In [ ]:
index = 13
title = valid_data[index][0]
body = valid_data[index][1]
generated_texts = generate_text_from_model(
    title=title, trained_model=trained_model, tokenizer=tokenizer, num_return_sequences=1
)
print('□ タイトル')
print(title)
print()
print('□ 生成本文')
print(generated_texts[0])
print()
print('□ 教師データ本文')
print(body)

□ タイトル
自転車女子、はじめましたvol.09「峠こそ、美女を作る!?」presented byゆるっとcafe

□ 生成本文
こんにちは、独女の皆様。お下劣、毒舌、ご満悦!ドロンジョーヌ恩田です。さて、自転車というさわやか丸出しの魅力に憑りつかれまして、その素晴らしさを広めるべく、鋭意活動中の三十路でございます。こればっかりは、老廃物が溜まってくる頃ではないでしょうか?空気が乾燥して身体が荒れたり、関節がブレたりすると、新陳代謝が悪くなって冷え性になったりします。夏は、暑いアスファルトの中を走るため、汗をかいたまま出歩くのもつらいでしょう。そして、そんな運動不足の毎日を少しでも長く乗り切るために、日焼け止めを塗って肌を露出するなんてことも大いにありそうです。それもそのはず。しかし、夏のレジャーシーズンに向け、愛車との長きに渡って愛用されるのが、富士フィルムのスタジオ「canon」。本格的な自転車テクニックには詳しくないので、いまさら聞けない人も多いと思います。ただ、峠道は大変だし、坂道はとにかくキツく走りたいもの。そこでオススメしたいのが、「峠こそ、美女を作る!」というもの。ひと漕ぎ目から飛び出す、静かな山道をザクザクのゆるふわしさ。水たまりやすい場所なんです。砂っぽかった街中で、突然ぶつかりあったり......。あの赤ちゃんみたいじゃなく、

□ 教師データ本文
今月もこんにちは。独女の皆様。お下劣、毒舌、即解決!ドロンジョーヌ恩田です。さて、常日頃から「坂道好き」を公言しているドロンジョーヌですが、やはり多くの人からすると、「なぜ、わざわざ自転車で坂道を?」と疑問に思われることでしょう。ドロンジョーヌだって坂道がキツいと思わないわけではありません。ただ、せっかく自転車で体を動かすなら、坂道を!と思ってしまいます。坂道を走るおもしろさや楽しさ、素晴らしさは、ほかに替えがたいものですから。登坂は、自分ひとりの体力と、自転車という道具だけで、自然へ挑む時間です。自分を信じ、ペースや呼吸を維持しながら、ただひたすら前へ進みます。体全体が軋むような苦しい時間だからこそ、坂道を上りきったときの達成感は最高です。空と自然に抱かれ、何ともいえない満足感。しかも、そのあとはスリルと緊張と高揚感を味わえる「下り坂」というオマケ付き!日常のストレスなんて、すべて坂の途中に置き